In [1]:
# import des librairies 
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle

# clean data
from transformers import BertTokenizer, BertForTokenClassification, BertForSequenceClassification
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

# charger le modèle 
from tensorflow.keras.models import load_model

2023-08-24 15:07:27.870657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 15:07:39.260775: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
jobs_df = pd.read_csv("../data/silver_data.csv")
# préparer un df avec uniquement les colonnes à utiliser pour le modèle
df = jobs_df[["jobs", "description", "ID_dep", "ville", "date", "experience", "skills", "tools", "industry", "company", "company_description"]]
df.head(3)

,jobs,description,ID_dep,ville,date,experience,skills,tools,industry,company,company_description
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75,PARIS 08,2023/07/12,5 ans,"concevoir et gérer un projet, concevoir un log...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75,PARIS 08,2023/07/12,5 ans,"analyser les résultats d'un projet, décliner l...",NaN,Conseil pour les affaires et autres conseils d...,QUANTMETRY,Pure player en Data et Intelligence Artificiel...
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92,ASNIERES SUR SEINE,2023/07/13,3 ans,contrôler et faire appliquer le respect de dis...,tableau,Activités des sièges sociaux,NEXITY,Nexity est aujourd hui leader sur les différen...


In [3]:
df = df[df['jobs'].notna()]
df.shape

(2613, 11)

In [ ]:
# Nettoyage de la colonne 'Jobs'
df['jobs'] = df['jobs'].str.replace(r"\(H/F\)", "", regex=True) \
                       .str.replace(r"\(F/H\)", "", regex=True) \
                       .str.replace(r"\(h/f\)", "", regex=True) \
                       .str.replace(r"\s*-?\s*CDI\s*", "", regex=True) \
                       .str.replace(r"\s*-?\s*CDD\s*", "", regex=True) \
                       .str.replace(r"\s*H/F\s*", "", regex=True) \
                       .str.replace(r"\s*F/H\s*", "", regex=True) \
                       .str.strip()

In [144]:
jobs_dirty = df["jobs"].tolist()
texts = jobs_dirty[2250:2300]
print(len(texts))
texts      

50


['Chef de projet informatique SI dans le domaine Data Center & Application Platform (IT) / Freelance',
 'consultant informatique (H/F/Y) Product Owner Data virtualisation DENODO (IT) / Freelance',
 'architecte informatique cloud data AWS Azure (IT) / Freelance',
 'DATA analyste informatiqueE BI SQL, BI4, SSIS, QlikSence Toulouse (IT) / Freelance',
 'consultant informatique(e) Test & recette informatique environnement Big Data (IT) / Freelance',
 'Tech Lead Big Data spark haddoop (IT) / Freelance',
 'ingénieur informatique Big Data (IT) / Freelance',
 'Data engeneer Google platform (IT) / Freelance',
 'Data Engineer Freelance - PySpark -- 2 jours sur site (IT) / Freelance',
 'Data Engineer Azure Data Factory senior Confirmé / Senior Paris 13ème (IT) / Freelance',
 'consultant informatique data migration SAP ADM (IT) / Freelance',
 'Data Center Architect - Freelance - Opportunités à distance (IT) / Freelance',
 'Data manager informatique (IT) / Freelance',
 'Data Engineer. Big Data, Scal

In [145]:
simplified_job_titles = [
    'Data Architect', 'Chef de projet', 'Data Engineer', 'PMO', 'Chef de projet', 'Chef de projet', 'Data Engineer', 'PMO', 'Data Scientist', 'Business Analyst', 'Business Analyst', 'Product Owner', 'Product Owner', 'Product Manager', 'Business Analyst', 'Data Analyst', 'Data Analyst', 'Data Analyst', 'Data Engineer', 'Chef de projet', 'Consultant', 'Data Scientist', 'Data Engineer', 'Data Engineer', 'Consultant', 'Data Analyst', 'Data Scientist', 'Data Architect', 'Data Engineer', 'Data Engineer', 'Data Architect', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Chef de projet', 'Data Analyst', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Scientist', 'Data Architect', 'Data Engineer', 'Data Analyst', 'Data Engineer', 'Data Architect', 'Data Engineer', 'Chef de projet',
    'Chef de projet',
     'Chef de projet',
     'Data Analyst',
     'Data Engineer',
     'Expert Data Engineering',
     'Techlead Data',
     'Ingénieur Data/Cloud',
     'Data Analyst',
     'Data Scientist',
     'Ingénieur Data',
     'Data Pricing Analyst',
     'Data Engineer',
     'Data Model',
     'Data Centre Architect',
     'Data Engineer',
     'Data Scientist',
     'Data Scientist/Engineer',
     'Développeur C#',
     'Senior Data Scientist',
     'Data Engineer',
     'Data Architect',
     'Consultant Data/BI',
     'Chef de projet Data',
     'Ingénieur Data',
     'Responsable Run Data',
     'Consultant Ops/Data/Cloud',
     'Data Engineer',
     'Senior Data Engineer',
     'Data Scientist',
     'Data Analyst',
     'Product Owner Data',
     'Chef de projet Data',
     'Data Warehouse',
     'Chef de projet Data',
     'Data Scientist',
     'Data Scientist',
     'Analyste Data',
     'Data Analyst/Developer',
     'Product Owner Data Management',
     'Data Engineer',
     'Data Analyst',
     'Data Engineer',
     'Chef de projet Open Data',
     'Data Manager',
     'Contrôleur de Gestion - Data Analyst',
     'Ingénieur Projet en Data Integrity',
     'Data Analyst',
     'Consultant Data RH',
     'Data Analyst',
     'Chef de projet MOA Data',
     'Data Manager',
     'Data Scientist',
     'Devops Big Data',
     'Data Analyst',
     'PMO Data',
     'Product Owner Data',
     'Data Analyst',
     'Data Analyst',
     'Contrôleur de Gestion - Data Analyst',
     'Data Engineer',
     'Juriste Data',
     'Ingénieur Data',
     'Data Manager',
     'Consultant Data',
     'Ingénieur Data Analyste',
     'Data Analyst',
     'Chef de projet Data Processing',
     'Data Engineer',
     'Data Engineer',
     'Manager Data',
     'Data Engineer',
     'Intégrateur Data',
     'Consultant Data Analytics',
     'Manager Data',
     'Consultant Data',
     'Chef de projet Data Processing',
     'Data Analyst',
     'Chef de projet Data',
     'Data Engineer',
     'Développeur Data',
     'Data Engineer',
     'Data Engineer',
     'Consultant Big Data Cloud',
     'Consultant MOE BI Data',
     'Data Analyst',
     'Analyste Développeur Data/BI',
     'Data Analyst E-Commerce',
     'Data Analyst',
     'Data Analyst',
     'Data Manager',
     'Ingénieur Mécanique Data',
     'Data Governance',
     'Data Scientist',
     'Software Engineer Big Data',
     'Chef de projet Data',
     'Consultant BI Data',
     'Data Engineer',
     'Consultant Data Visualisation',
     'Consultant Data Migration',
     'Data Engineer',
     'Chef de projet Data Gouvernance',
     'Chef de projet Data',
     'Expert Big Data',
     'Responsable DBA',
     'Data Engineer',
     'Project Manager Data',
     'Solution Engineer Data',
     'Data Engineer',
     'Data Analyst',
     'Architecte Data',
     'Consultant Data',
     'Développeur Big Data',
     'Data Engineer',
     'Développeur Big Data',
     'Master Data',
     'Data Analyst',
     'Data Web Analyst',
     'Architecte Data',
     'Data Engineer',
     'Data Ingénieur',
     'Data Analyst',
     'BPM Data',
     'Business Analyst',
     'Business Analyst',
     'Data Analyst',
     'Data Architect',
     'Data Engineer',
     'Chef de projet Data',
     'Consultant Data',
     'Data Analyst',
     'Data Analyst',
     'Responsable Data',
     'Développeur Data',
     'Consultant Data',
     'Data Analyst',
     'Data Architect',
     'Data Scientist',
     'Data Analyst',
     'Data Engineer',
     'Data Engineer',
     'Architecte Data',
     'Data Scientist',
     'Ingénieur Data',
     'Data BA',
     'Ingénieur Data',
     'BA Data',
     'Data Steward',
     'Data Ingénierie',
     'PMO/Data Analyst',
     'Data Analyst',
     'Tech Lead Data Engineer',
     'Data Architect',
     'Directeur de Projet Data',
     'Chef de Projet Big Data',
     'Data Engineer',
     'Data Manager',
     'Data Manager',
     'Product Owner Data',
     'Data Architect',
     'Consultant Big Data',
     'Chef de Projet Data',
     'Responsable Master Data',
     'Data Engineer',
     'Contrôleur de Gestion - Data Analyst',
     'Ingénieur Data',
     'Data Engineer Alternant',
     'Marketing Data Manager',
     'Data Analyst',
     'Data Manager',
     'Consultant Data Engineer',
     'Consultant Data Visualisation',
     'Data BI Architecte',
     'Manager Data Contrôleur',
     'Data Analyst - PowerBI',
     'CRM - Data Analyst',
     'Chef de Projet Data Center',
     'Développeur Big Data',
     'Data Engineer',
     'Data Engineer Cloud',
     'Data Analyst',
     'PMO - Data Analyst',
     'Data Engineer Spark - Scala',
     'Manager Data',
     'Consultant Data Cloud',
     'Data Architect',
     'Data Engineer',
     'Data Engineer',
     'Expert Data Gouvernance',
     'Ingénieur Projet Data Integrity',
     'Data Contrôleur',
     'Data Engineer Spark - Scala',
     'Architecte Data Cloud',
     'Data Analyst Indemnisation',
     'Data Analyst',
     'Expert Conseil Data',
     'Data Contrôleur',
     'Data Manager',
     'Développeur Big Data',
     'Consultant Data BI & Analytics',
     'Développeur Big Data',
     'Développeur Big Data',
     'Ingénieur DATA',
     'Consultant Data Analyst Power BI',
     'Consultant Senior Data Gouvernance / MDM',
     'Data Analyst Qlik Sense',
     'Responsable Pôle Data et Méthodes',
     'Data Manager',
     'Data Manager',
     'Développeur Big Data',
     'Consultant Data Azure',
     'Data Scientist',
     'Data Analyst',
     'DevOps Data',
     'Data Scientist',
     'Data Engineer',
     'Data Scientist',
     'Analyste Normes Data',
     'Développeur Big Data',
     'Data Manager',
     'Développeur Big Data',
     'Data Scientist',
     'Développeur Big Data',
     'Développeur Big Data',
     'Data Engineer Cloud - Google Cloud',
     'Expert Technique CVC DI Data Center',
     'Développeur Big Data',
     'Chef de Projet Data',
     'Data Manager',
     'Développeur Big Data',
     'Data Analyst',
     'Data Manager',
     'Data Analyst',
     "Chargé d'Affaires Électrique et Data Center",
     'Data Analyst Approvisionnement',
     'Consultant Big Data',
     'Data Scientist',
     'Data Manager',
     'Data Scientist',
     'Chef de Projet Data',
     'Data Marketing Alternance',
     'Développeur Big Data',
     'Développeur Web - Data Management',
     'Software Engineer Big Data',
     'Scrum Master Transformation DATA',
     'Développeur Big Data',
     'Développeur Big Data',
     'Consultant Senior Data',
     'Responsable Data',
     'Chef de Projet Data',
     'Data Manager',
     'Data Analyst',
 'Chef de Projet Data',
 'Data Project Manager',
 'Data Analyst',
 'Data Analyst',
 'Data Manager',
 'Data Analyst',
 'Data Manager',
 'Data Analyst / BI QLik',
 'Ingénieur Data',
 'Product Owner Data',
 'Data Scientist',
 'Data Manager',
 'Data Scientist',
 'Data Analyst',
 'Data Manager',
 'Data Manager',
 'Manager Data & Stratégie',
 'Manager Data',
 'Assistant Data Marketing',
 'Développeur ETL',
 'Développeur Big Data',
 'Data Manager',
 'Développeur Big Data',
 'Data Analyst',
 'Data Analyst - Secteur Énergie',
 'Data Scientist/Statistician',
 'Expert Big Data',
 'Développeur Data',
 'Développeur MDM',
 'Data Manager',
 'Data Analyst',
 'Data Privacy & Protection Analyst',
 'Data Analyst Volume',
 'Data Analyst',
 'Expert Big Data',
 'Data Analyst',
 'Data Analyst',
 'Architecte DATA',
 'Data Scientist/Analyst',
 'Développeur Big Data',
 'Data Scientist',
 'Assistant Marketing & Data',
 'Product Owner Data',
 'Développeur Big Data',
 'R&D Data Analyst',
 'Développeur Big Data',
 'Data Manager',
 'Manager Data',
 'Data Manager',
    'Data Engineer',
 'Data Manager',
 'Développeur Big Data',
 'Data Scientist',
 'Manager Investissements & Empreinte Carbone',
 'Développeur Big Data',
 'Data Analyst',
 'Ingénieur Supply Chain - Data Science',
 'Développeur Big Data',
 'Développeur Big Data',
 'Data Manager',
 'Responsable Data',
 'Ingénieur Hospitalier - Adjoint DPO',
 'Chef de Projet - Accreditation Data',
 'Chargé de Mission Data Visualisation',
 'Dessinateur-Projeteur Mécanique',
 'Développeur Big Data',
 'Technicien Data',
 'Ingénieur Data',
 'Data Manager',
 'Développeur Big Data',
 'Scrum Master Data',
 'Data Manager',
 'Data Analyst',
 'Data Scientist',
 'Data Manager',
 'Data Scientist',
 'Dessinateur Projeteur DATA CENTER',
 'Data Engineer',
 'Data Scientist',
 'Chargé de Projet Data Visualisation',
 'Data Contrôleur - Assistant Comptable',
 'Alternance Data Engineer',
 'Chef de Projets Data Management',
 'Consultant Ingénieur des Données',
 'Data Contrôleur - Assistant Comptable Agricole',
 'Ingénieur Data Intégration',
 'Ingénieur de Sûreté de Fonctionnement',
 'Chef de Projets Master Data & Supply Chain',
 'Data & Business Analyst',
 'Chef de Projet ETL',
 'Data Analyst (Telecom)',
 'Data Analyst - Snowflake & Power BI',
 'Senior Data Engineer',
 'Expert Data',
 'Chargé de Pilotage Opérationnel',
 'Développeur Big Data',
 'Formation Technicien Data Analyst',
 'Data Engineer',
 "Responsable Système d'Information & DATA LIA",
 'Consultant BI',
 'Data Analyst',
 'Data Engineer',
 'Data Analyst',
 'Data Engineer',
 'Data Analyst',
 'Data Engineer',
 'Business Analyst',
 'Apprenti Environnement et Climat',
 'Ingénieur Cloud',
 'Ingénieur de Travaux',
 'Product Owner CRM',
 'Data Analyst',
 'Technicien Data',
 'Technicien Data Center',
 'Assistant Comptable',
 'Data Engineer',
 'Ingénieur développeur Big Data',
 'Chef de projet informatique',
 'Data Analyst',
 'Data Scientist',
 'Business Analyst',
 'Assistant Data Produit',
 'Data Analyst',
 'Contrôleur de données',
 'Data Analyst',
 'Data Analyst',
 'Référent technique',
 'Data Analyst',
 'Data Analyst',
 'Collaborateur comptable',
 'Chargé de data patrimoniales',
 'Consultant Protection des Données',
 'Data Analyst',
 'Data Manager',
 'Assistant Comptable',
 'Développeur Data Scientist',
 'Data Engineer',
 'Documentaliste technique',
 'Consultant Data Scientist',
 'Product Owner Data',
 'Data Scientist',
 'Product Owner Data',
 'Product Owner Data',
 'Manager Data Scientist',
 'Product Owner Data',
 'Product Owner Data',
 'Product Owner Data',
 'Data Scientist',
 'Data Analyst',
    'Quality Analyst',
 'Ingénieur projet Data Center',
 'Product Owner Data',
 'Product Owner Data',
 'Data Analyst',
 'Analyste Applicatif Data',
 'Responsable CRM',
 'Ingénieur Data',
 'Product Owner Data',
 'Consultant Data Analytics',
 'Opérateur de saisie',
 'Data Analyst',
 'Product Owner Data',
 'Product Owner Data',
 'Data Manager',
 'Data Analyst',
 'Product Owner Data',
 'Data Manager',
 'Data Manager',
 'Product Owner Data',
 'Data Analyst',
 'Product Owner Data',
 'Product Owner Data',
 'Développeur Master Data',
 'Data Analyst',
 'Data Manager',
 'Product Owner Data',
 'Data Analyst',
 'Data Analyst',
 'Product Owner Data',
 'Product Owner Data',
 'Chef de projet SI Retail',
 'Data Base Administrateur',
 'Architecte Big Data',
 'Architecte Big Data',
 'Chef de Projets Techniques ERP',
 'Product Owner Data',
 'Data Analyst',
 'Data Analyst',
 'Product Owner Data',
 'Data Analyst',
 'Développeur Back-End',
 'Data Scientist',
 'Product Owner Data',
 'Consultant Data Science',
 'Data Analyst',
 'Data Analyst',
 'Data Engineer',
 'Consultant Data BI',
 'Consultant Data BI',
   'Data contrôleur',
 'Data Engineer',
 'PMO Tech Data',
 'Data Analyst',
 'Développeur Big Data',
 'Consultant Data',
 'Technicien data center',
 'Architecte Big Data',
 'Data Scientist',
 'Data Analyst',
 'Data Scientist NLP',
 'Data Scientist',
 'Consultant Data',
 'Sales Senior Data',
 'Consultant Data',
 'Data Scientist',
 'Data Analyst',
 'Ingénieur Big Data',
 'Data Analyst',
 'Data Scientist',
 'Consultant Data',
 'Data Engineer',
 'Data Analyst',
 'Technicien Data',
 'Data Integrity Analyst',
 'Data Engineer',
 'Data Engineer',
 'Consultant Data/BI',
 'Juriste Assistant DPO',
 'Data Analyst',
 'Chef de projet informatique ETL',
 'Ingénieur statisticien',
 'Data Analyst',
 'Data Analyst',
 'Data Manager',
 'Data Analyst',
 'Développeur Big Data',
 'Architecte Big Data',
 'Data Engineer',
 'Data Engineer',
 'Data Engineer',
 'Data Engineer',
 'Business Analyst',
 'Gestionnaire de données',
 'Développeur IA',
 'Data Ingénieur',
 'Gestionnaire de données',
 'Data Analyst',
 'Financial Data Analyst',
 'Ingénieur Data', 
  'Ingénieur Data BI',
 'Data Manager',
 'Data Analyste Risques',
 'Ingénieur Big Data',
 'Ingénieur Data',
 'Assistant Chef de Projet Data CRM',
 'Manager Data',
 'Data Scientist',
 'Data Engineer',
 'Chef de Projets Data Décisionnel',
 'Data Analyst',
 'Data Scientist',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Steward',
 'Data Engineer',
 'Data Scientist',
 'Data Engineer',
 'Data Engineer',
 'Data Engineer',
 'Data Engineer',
 'Data Analyst',
 'Data Engineer',
 'Data Analyst',
 'Data Analyst',
 'Data Scientist',
 'Business Analyst IT',
 'Product Owner Data',
 'Data Scientist',
 'Chef de Projet Infrastructure Data Center',
 'Développeur Data SQL',
 'Développeur C# Linux Embarqué',
 'Data Scientist',
 'Data Scientist',
 'Développeur Big Data',
 'Assistant Chef de Projet Data CRM',
 'Data Analyst',
 'Manager Data Scientist',
 'Assistant Marketing Data Management',
 'AMOA Business Analyst Data',
 'Architecte Réseaux Data Center',
 'Data Analyste- Contrôleur de Gestion',
 'Développeur Python Big Data',
 'Data Analyst',
 'Proxy Product Owner Big Data',
 'Data Analyst',
 'Développeur Java',
 'Data Scientist',
 'Expert Réseaux',
 'Data Manager',
 'Assistant Chef de Projet',
 'Data Analyst',
 'Data Analyst',
 'Data Manager',
 'Product Owner',
 'Data Analyst',
 'Product Owner',
 'Product Owner',
 'Directeur de Projet',
 'Assistant Marketing',
 'Consultant BI',
 'Product Owner',
 'Data Analyst',
 'Chef de Projet',
 'Data Scientist',
 'Data Analyst',
 'Project Manager',
 'Product Owner',
 'Data Scientist',
 'Product Owner',
 'Product Owner',
 'Product Owner',
 'Product Owner',
 'Consultant MOA',
 'Data Analyst',
 'Technicien SAP',
 'Directeur Data',
 'Product Owner',
 'Chef de Projet',
 'Architecte Big Data',
 'Développeur Big Data',
 'Product Owner',
 'Développeur Data',
 'Ingénieur Big Data',
 'Product Owner',
 'Data Analyst',
 'Data Scientist',
 'Développeur Big Data',
 'Data Ingénieur',
 'Data Engineer',
 'Expert Conseil',
 'Data Analyst',
 'Gestionnaire ADV',
 'Développeur Data',
 'Data Analyst',
    'Data Analyst', 'Data Analyst', 'Data Analyst', 'Product Owner', 'Data Engineer', 'Chef de Projet', 'Consultant Data Cloud', 'Chef de Projet SIG', 'Data Architect', 'Data Analyst', 'Chef de Projet Data', 'Développeur Big Data', 'Architecte Big Data', 'Data Analyst SIRH', 'Consultant Data & BI', 'Consultant Data', 'Développeur Java Big Data', 'Tech Lead/Data Architect', 'Data Engineer', 'Data Analyst', 'Architecte Cloud Data', 'Data Center Architect', 'Product Owner Data Virtualisation', 'Consultant Test & Recette Big Data', 'Commercial Btob Data', 'Data Analyst', 'Business System Analyst', 'Chef de Projet IT Data Center', 'Data Engineer', 'Architecte Data Hadoop', 'Data Steward', 'Développeur Big Data', 'Data Engineer', 'Data Scientist', 'Data Engineer', 'Data Manager', 'Développeur Big Data', 'Technicien CVC Data Center', 'Data Analyst', 'Data Engineer', 'Développeur Big Data', 'Expert Data', 'Chef de Projet IT Data', 'Assistant Marketing/Data Analyst', 'Scrum Master', 'Assistant Data Analyst', 'Développeur Big Data', 'Data Scientist', 'Data Analyst', 'Ingénieur de Travaux Data Center',
    'Business Analyst', 'PMO Data', 'Data Engineer', 'Ingénieur Data', 'Data Analyst', 'Ingénieur Data', 'Architecte Data BI', 'Data Scientist', 'Data Engineer', 'Chef de Projet Data Center & Application Platform', 'Data Scientist', 'Data Analyst', 'Data Analyst', 'Data Analyst/Scientist', 'Data Architect', 'Directeur de Projet Data', 'Data Analyst Power BI', 'Data Scientist', 'Data Analyst Junior', 'Data Scientist', 'Data Manager', 'Data Scientist', 'Data Manager', 'Tech Lead Data', 'Tech Lead Big Data', 'Proxy PO Data', 'Developpeur Data ESG', 'Data Engineer Azure', 'Developer Data PL-SQL/Shell/BO', 'Consultant Data Power BI', 'Data Protection Officer', 'Data Engineer', 'Data Engineer', 'Data Engineer Spark', 'Développeur Big Data', 'Professeur Data RH', 'Senior Data Scientist', 'Data Scientist', 'Tech Lead Data', 'Ingénieur Big Data', 'Manager', 'Data Analyst', 'Data Manager', 'Data Analyst Support', 'Data Scientist', 'Data Analyst', 'Data Manager', 'Développeur Big Data', 'Data Manager', 'Chef de Projet Data',
  'Data Scientist',
 'Développeur Python',
 'Data Scientist',
 'Chef de Projet Big Data',
 'Architecte Big Data',
 'Data Scientist/Data Analyst',
 'Consultant Data Scientist',
 'Data Engineer',
 'Data Engineer',
 'Data Analyst',
 'Gestionnaire de Données',
 'IS Support Analyst',
 'Data Engineer',
 'Responsable Produit Data RH',
 'Chef de Projet Data Center',
 'Analyste Applicatif Data',
 'Consultant Data Migration',
 'Chef de Projet Data',
 'Pricing Data Analyst',
 'Ingénieur Recherche Deep Learning',
 'Ingénieur DevOps Data',
 'Expert Data',
 'Data Scientist',
 'Data Scientist',
 'Data Center Architect',
 'Data Engineer GCP BigQuery',
 'Consultant Data Scientist',
 'Chargé de Recette Data',
 'Chef de Projet Data',
 'Data Contrôleur',
 'Développeur Big Data',
 'Chef de Projets Data',
 'Data Manager',
 'Data Scientist',
 'Data Steward',
 'Data Engineer Senior',
 'Commercial Btob Data',
 'Business Analyst Master Data',
 'Architecte Data Stratégie',
 'Data Protection Correspondent',
 'Consultant Data IBM InfoSphere',
 'Data Scientist',
 'Senior Data Engineer',
 'Commercial Btob Data',
 'Développeur Talend',
 'Consultant Data',
 'Chef de Projets Data Décisionnel',
 'Data Scientist/Analyst',
 'Ingénieur DevOps/Big Data/Cloud',
 'Data Analyst',
   'Consultant Data Scientist',
 'Ingénieur Big Data',
 'Data Architect',
 'Technicien Data Center',
 'Commercial Btob Data',
 'Consultant Data',
 'Product Owner Data',
 'Chef de Pôle Animation Data',
 'Développeur Big Data',
 "Chargé d'Animation Data",
 'Data Analyst',
 'Data Analyst',
 'Commercial Btob Data',
 'Data/Dataviz Developer',
 'Consultant Data MDM',
 'Data Analyst',
 'Data Scientist',
 'Consultant Data Scientist',
 'Data Analyst',
 'Data Steward',
 'Chef de Projets Data et Innovation',
 'Data Management GCP',
 'Data Protection Correspondent',
 'Développeur Python Data',
 'Talend Data Engineer',
 'Data Scientist',
 'Développeur Big Data',
 'Ingénieur Réseaux Data Center',
 'Data Architect GCP',
 'Senior Data Engineer',
 'Business Analyst Data Governance',
 'Administrateur Plateforme Data',
 'Business Analyst Data',
 'Data Engineer',
 'Chef de Projet SAP Data Migration',
 'Data Engineer Big Data',
 'Data Engineer Dataiku',
 'Data Engineer',
 'Data Manager',
 'Développeur BI/BIG DATA',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Technicien Data Center',
 'Technicien Data Center',
 'Ingénieur DATA',
 'Analyste Master Data',
 'Data Engineer',
 'Data Engineer Junior',
 'Product Owner Data',
    'Architecte Data',
 'Responsable Projet Data',
 'Chef de Projet Data',
 'Data Analyst',
 'Data Analyst',
 'Consultant Data',
 'Data Marketing',
 'Consultant Data Gouvernance',
 'Expert Data & IA',
 'Data Architect',
 'Data Architect',
 'Tech Lead Data',
 'Data Analyst',
 'Data Engineer',
 'Architecte Big Data',
 'Consultant Data Gouvernance',
 'Développeur Data AWS',
 'Data Product Owner',
 'Data Analyst',
 'Pilote de Projet Data',
 'Développeur Big Data',
 'Consultant Data Gouvernance',
 'Chef de Projet Data',
 'Architecte Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Analyste Master Data',
 'Data Manager',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Data Manager',
 'Data Manager',
 'Chef de Projet Mes/Data',
 'Développeur Big Data',
 'Développeur Big Data',
 "Chargé d'Animation Data",
 'Chef de Projet ERP - Data',
 'Développeur Big Data',
 'Assistant Data',
 'Data Engineer',
 'Data Analyst Qlik Sense',
 'Manager Système & Outils Data',
 'Assistant Marketing & Data',
 'Data Engineer Python SQL',
 'Manager',
 'Chef de Pôle Animation Data',
 'Chef de Projets Data & Innovation',
    'Data Analyst',
 'Ingénieur Big Data',
 'Data Engineer',
 'Data Manager',
 'Data Analyst R&D',
 'Développeur Big Data',
 'Tech Lead Data',
 'Data Scientist',
 'Manager Data',
 'Data Analyst',
 'Data Scientist/Analyst',
 'Data Analyst',
 'Assistant Data Financière',
 'Chef de Projet BI & Data',
 'Web Analyst',
 'Data Analyst DSI Bancaire',
 'Data Steward',
 'Data Analyst CRM',
 "Chef d'Unité Data",
 'Data Scientist',
 'Data Manager',
 'Data Analyst',
 'Business Analyst Data',
 'Data Analyst IOT/BI',
 'Data Analyst',
 'Manager Data Wranglers',
 'Data Manager',
 'Data Engineer',
 'Ingénieur Data',
 'Chef de Projet DATA',
 'Data Product Owner',
 'Ingénieur Data SQL Server',
 'Data Quality Administrator',
 'Data Engineer GCP/SQL',
 'Data Engineer',
 'Consultant Data Gouvernance/MDM',
 'Data Project Manager',
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
    'Développeur Data/ETL',
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 'Technicien Data Center',
 'Data Scientist/Analyst',
 'Ingénieur Data',
 'Lead Data Scientist',
 'Data Engineer Sénior',
 "Directeur d'Agence RGPD""Directeur d'Agence RGPD",
 'Data Consultant Stratégie',
 "Directeur d'Agence RGPD",
 'Consultant Data Architect',
 "Directeur d'Agence RGPD",
 'Business Analyst',
 "Directeur d'Agence RGPD",
 'Business Analyst',
 'Business Analyst',
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 'Business Analyst',
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 'Technicien Data Center',
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
 "Directeur d'Agence RGPD",
    "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Data Analyst',
 "Directeur d'agence (RGPD)",
 'Data Scientist',
 "Directeur d'agence (RGPD)",
 'Juriste',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Gestionnaire de données',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Responsable Informatique',
 'Data Analyst',
 'Responsable des Outils - Projets - Data Finance',
 'Directeur Data',
 'Data Analyst',
 'Ingénieur Data',
    'Data Manager',
 'Data Analyst',
 'Expert Réseaux',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Data Analyst',
 'Responsable Informatique',
 'Product Owner',
 'Data Analyst',
 'Développeur Python',
 'Data Analyst',
 'Product Owner',
 'Architecte Réseaux',
 'Développeur Data',
 'Data Analyst',
 'Data Scientist',
 'Data Scientist',
 'Data Analyst',
 'Analyste Développeur',
 'Chef de Projets',
 'Data Analyst',
 'Directeur Data',
 'Clinical Data Manager',
 'Data Analyst',
 'Product Owner',
 'Data Analyst',
 'Data Analyst',
 'Data Manager Senior',
 'Data Analyst',
 'Chef de Projet',
 'Chef de Projets',
 'Product Owner',
 'Data Analyst',
 'Product Owner',
 'Data Scientist',
 'Product Owner',
 'Data Manager',
 'Apprenti Data Analyst',
 'Ingénieur Data',
 'Ingénieur Big Data',
 'Data Engineer',
 'Data Analyst',
 'Coordinateur Approvisionnements',
 'Data Analyst',
 'Data Scientist',
 'Data Analyst',
    'Chef de Projet',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Engineer',
 'Consultant Big Data',
 'Data Architect',
 'Ingénieur Big Data',
 'Tech Lead BI & Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 'Data Engineer',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Expert Data Finance',
 'Développeur BI',
 'Chef de Projet Infrastructure',
 'Chef de Projet Data',
 'Assistant Chef de Projet Data',
 'Data Engineer',
 'Data Scientist',
 'Data Analyst',
 'Ingénieur Informatique',
 'Data Analyst',
 'Consultant Senior Data Scientist',
 'Data Analyst',
 'Data & Reporting Analyst',
 'Data Analyst',
 'Technicien Data',
 'Data Ingénieur',
 'Consultant Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Gestionnaire Data',
 'Data Analyst',
 'Ingénieur Java & Data Integration',
 'Data Analyst',
 'Data Scientist',
 'Data Analyst',
 'Data Analyst IoT',
 'Expert Cloud Data',
 "Chargé d'études Data",
 'Data Analyst',
 'Data Analyst',
 'Data Engineer',
 'Data Analyst',
    'Data Analyst',
 'Analyste Développeur Data Scientist',
 'Data Analyst',
 'Data Visualization Officer',
 'Data Analyst',
 'Data Analyst',
 'Assistant Chef de Projet Data',
 'Data Analyst',
 'Ingénieur Data Integrity',
 'Consultant Data Gouvernance',
 'Data Analyst Marketing',
 'Scrum Master Data',
 'Data Analyst',
 'Chef de Projet Data',
 'Ingénieur Big Data',
 'Gestionnaire de Data Center',
 'Ingénieur Développeur Data',
 'Analyste Développeur Data',
 'Data Analyst',
 'Chargé de Mission ERP et DATA',
 'Data Engineer',
 'Data Analyst',
 'Développeur Big Data',
 'Data Analyst',
 'Data Analyst Power BI',
 'Développeur Big Data',
 'Chef de Projet Technique Data',
 'Data Analyst',
 'Data Analyst',
 'Ingénieur Développement IA',
 'Lead Data Engineer',
 'Correspondant Protection des Données',
 'Adjoint DPO',
 'Analyste Applicatif Data',
 'Apprenti Informaticien Data Scientist',
 'Data Engineer',
 'Leader Technique Data BI',
 'Administrateur Réseaux DATA',
 'Data Scientist',
 'Data Analyst RH',
 'Data Scientist',
 'Data Engineer',
 'Consultant Data Scientist',
 'Data Analyst',
 'DBA Développeur SQL',
 'Data Scientist',
 'Data Scientist Senior',
 'Data Engineer',
 'Contrôleur de Gestion Data Analyst',
 'Assistant Data Marketing',
    'Data Engineer',
 'Data Manager',
 'Data Analyst',
 'Data Manager',
 'Data Analyst',
 'Data Manager',
 'Data Analyst CRM',
 'Data Analyst Power BI',
 'Ingénieur Data',
 'Data Analyst',
 'Analyste Audit IT & Data',
 'Data Engineer',
 'Data Scientist',
 'Data Analyst',
 'Responsable Data Logistique',
 'Chargé de Support Applicatif Data',
 'Développeur Big Data',
 'Gestionnaire Master Data',
 'Data Engineer',
 'Développeur Back-End & Data',
 'Data Analyst',
 'Assistant Data Scientist',
 'Data Scientist',
 'Assistant Comptable Data',
 'Consultant BI',
 'Assistant Data Analyste',
 'Data Scientist',
 'Responsable Système',
 'Technicien Conception Systèmes',
 'Assistant Marketing Digital',
 'Data Engineer',
 'Data Scientist',
 'Data Architect',
 'Consultant Data',
 'Consultant Data Gouvernance',
 'Ingénieur Data BI',
 'Data Analyst',
 'Data Engineer',
 'Architecte Big Data',
 'Consultant Data Gouvernance',
 'Manager Data',
 'Chef de Projet Data',
 'Data Engineer',
 'Data Engineer',
 'Consultant Data Analyste',
 'Consultant BI',
 'Consultant Data',
 'Responsable Data Science',
 'Lead Data Manager',
 'Lead Data Science',
    'DBA Développeur',
 'Data Scientist',
 'Data Visualization Officer',
 'Architecte Plateforme',
 'Tech Lead',
 'Consultant Architecte',
 'Data Quality Manager',
 'Consultant Data',
 'Gestionnaire des Masters Data',
 'Data Analyst',
 'Data Scientist',
 'Data Engineer',
 'Data Architect',
 'Consultant Data',
 'Développeur Master Data Management',
 'Data Manager',
 'Data Analyst',
 'Finance Data And Controlling',
 'Data Manager',
 'Data Ingénieur',
 'Data Analyst',
 'Data Analyst',
 'Data Manager',
 'Data Engineer',
 'Développeur Big Data',
 'Data Engineer',
 'Data Manager',
 'Data Analyst',
 'Administrateur Ministériel',
 'System Team Engineer',
 'Ingénieur Développeur',
 'Technicien Informatique',
 'Technicien Gestionnaire de Données',
 'Data Scientist',
 'Développeur Full Stack',
 'Data Engineer',
 'Analyste Data',
 'Tech Lead',
 'Data Manager',
 'Consultant Data',
 'Data Scientist',
 'Consultant Data',
 'Charge de Data Management',
 'Data Analyst',
 'Analyste Data',
 'Actuaire/Data Scientist',
 'Data Architect',
 'Data Analyst',
 'Data Scientist',
 'Data Scientist',
    'Data Analyst',
 'Consultant Data',
 'Data Steward',
 'Data Manager',
 'Chef de Projet Data',
 'Data Manager',
 'Directeur de Projet Data',
 'Chef de Projet Data',
 'Data Analyst',
 'Data Analyst',
 'Data Manager',
 'Data Scientist',
 'Data Manager',
 'Data Manager',
 'Responsable de Dossiers',
 'Data Analyst',
 'Data Scientist',
 'Ingénieur Big Data',
 'Architecte Data',
 'Data Engineer',
 'Chef de Projet Data',
 'Juriste Droit des Contrats',
 'Chief Data Officer',
 'Administrateur Réseaux',
 'Data and Business Analyst',
 'Ingénieur Data',
 'Data Analyst',
 'Data Analyst',
 'Chef de Service Publication',
 'Data Manager',
 'Data Analyst',
 'Data Manager',
 'Chef de Projet Data',
 'Ingénieur Informatique',
 'Architecte Data',
 'Data Analyst',
 'Data Analyst',
 'Ingénieur Développeur',
 'System Team Engineer',
 'Data Analyst',
 'Data Engineer',
 'Correspondant Protection des Données',
 'Responsable CRM & Data',
 'Analyste Fonctionnel',
 'Ingénieur Data',
 'Aide Électricien',
 'Consultant Manager Data',
 'Directeur de Projet Data',
 'Data Manager',
 'Manager Data Scientist',
    'Data Analyst',
 'Ingénieur Data',
 'Assistant à la Protection des Données',
 'Consultant Data Analytics',
 'Data Engineer',
 'Chef de Projet Data',
 'Data Architect',
 'Consultant Data',
 'Data Engineer',
 'Développeur Master Data Management',
 'Ingénieur Data Integrity',
 'Data Manager',
 'Administrateur Réseaux Data',
 'Analyste Data et CRM',
 'Data Analyst',
 'Data Architect',
 'Data Officer',
 'Data Officer',
 'Data Analyst',
 'Actuaire/Data Scientist',
 'Data Engineer',
 'Data Officer',
 'Ingénieur Data',
 'Développeur Full Stack Big Data',
 'Data Engineer',
 'Chef de Projet Data/BI',
 'Chargé de Validation Projet Data',
 'Data Analyst',
 'Data Architect',
 'Data Architect',
 'Data Engineer Azure',
 'Data Analyst',
 'Chargé de Validation Projet Data',
 'Data Analyst',
 'Tech. Lead Big Data',
 'Data Analyst',
 'Data Architect',
 'Data Analyst',
 'Data Marketing Analyste',
 'Senior Data Analyst',
 'Data Engineer',
 'Consultant Data',
 'Consultant TALEND Big Data',
 'Data Engineer',
 'Chef de Projet Data',
 'Analyste DATA',
 'Data Analyst Power BI',
 'Data Engineer',
 'Clinical Data Manager',
 'Senior Data Manager',
    'Data Scientist',
 'Chef de Projet Data',
 'Data Engineer',
 'Business Analyst Data',
 'Développeur SQL - Data Manager',
 'Data Engineer',
 'Product Owner Data',
 'Data Analyst',
 'Global Master Data Supply Chain',
 'Consultant Data-Bi',
 'Data Engineer',
 'Data Engineer',
 'Data Scientist',
 'Senior Data Analyst',
 'Data Analyst',
 'Responsable de Dossiers - Data Controleur',
 'Data Analyst',
 'Data Engineer',
 'Consultant Data Intégration',
 'Data Engineer',
 'Data Analyst Power Bi',
 'Chef de Projet Data',
 'Employé Production - Data Quality',
 'Data Analyst',
 'Data Analyste Performance Commerciale',
 'Data Engineer Big Data',
 'Data Engineer Exploitation',
 'Data Engineer Big Data',
 'Data Engineer',
 'Ingénieur Data Science',
 'Ingénieur Data Science',
 'Consultant Senior Data',
 'Responsable Méthodes et Process Data Management',
 'Expert Data Privacy',
 'Business Analyst IT',
 'Business Analyst Risques de Marché',
 'Ingénieur de la Donnée',
 'Chef de Section',
 'Data Scientist',
 'Business Analyst Stress Tests',
 'Master Data Gestionnaire',
 'Chef de Projet Data',
 'Data Manager',
 'Business Analyst',
 'Data Advisory Senior',
 'Développeur Master Data Management',
 'Data Engineer/MLOPS',
 'Data Scientist',
 'CP Data',
 'Consultant Data Engineer',
 'Business Analyst',
 'Data Analyst',
 'Architecte Data',
 'Ingénieur Data Science',
 'Développeur Master Data Management',
 'Ingénieur Data Science',
 'Data Manager',
 'Business Analyst',
 'Data Scientist',
 'Data Scientist',
 'Data Manager',
 'Data Analyst',
 'Développeur Big Data',
 'Data Scientist',
 'Développeur Big Data',
 'Data Analyst',
 'Ingénieur Big Data',
 'Data Analyst',
 'Data Manager',
 'Data Manager',
 'Data Manager',
 'Développeur Big Data',
 'Développeur Big Data',
 'Ingénieur DevOps Big Data',
 'Data Manager',
 'Ingénieur Data',
 'Ingénieur Informatique',
 'Chargé de Marketing Data',
 'Chargé de Projet SIG et Open Data',
 'Ingénieur Informatique',
 'Data Manager',
 'Data Manager',
 'Data Scientist',
 'Data Manager',
 'Data Manager',
 'Ingénieur Data',
 'Proxy Product Owner',
 'Product Owner Master Data Management',
 'Développeur Talend',
 'Consultant Power BI Data Analyst',
 'Data Scientist',
 'Référent Applicatif & Data',
 'Data Steward',
 'Data Analyst',
 'Manager BI/Data',
 'Data Engineer',
 'Data Analyst',
 'Chef de Projet Data Services',
 'Consultant AMOA SIRH DATA',
 'Data Manager Supply Chain',
 'Data Analyst/Scientist',
 'Data Manager',
 'Chef de Mission Comptable',
 'Data Steward',
 'Data Architecte Azure/Databricks/Cassandra',
 'Collaborateur Comptable Confirmé',
 'Data Steward',
 'Data Quality Manager',
 'Data Manager',
 'Data Protection Officer',
 'Responsable Master Data Produit',
 'Data Engineer',
 'Consultant Stratégie Data et IA',
 'Data and Business Analyst',
 'Data Analyst Alteryx',
 'Data Engineer Azure',
 'Gestionnaire Data SQL Junior',
 'Data Engineer',
 'Ingénieur Big Data Talend',
 'Data Engineer',
 'Consultant Business Analyst Data',
 'Data CRM/Manager',
 'Chef de Projet Pilotage Data IA',
 'Data Engineer Big Query GCP Kafka',
 'Data Analyst',
 'Consultant Data Management',
 'Data Scientist',
 'Data Ops Azure',
 'Data Architecte',
 'Data Marketing',
 'Data Marketing',
 'Consultant Data Visualisation',
 'Référent Data',
 'Manager Data Development',
 'Consultant Data BI',
 'Lead Data Scientist',
 'Gestionnaire Middle Office',
 'Chef de Projet Data',
 'Responsable Data',
 'Chef de Projet Data',
 'Développeur Big Data Django/Angular',
 'Gestionnaire Master Data',
 'Ingénieur Big Data Confirmé',
 'Chef de Projet Data Intelligence',
 'Project Manager Data',
 'Data Engineer',
 'Big Data Engineer',
 'Ingénieur Big Data Confirmé',
 'Ingénieur Recherche Deep Learning',
 'Ingénieur Data Analyst',
    'Consultant MOA Big Data',
 'Data Engineer',
 'QA Data',
 'Data Engineer',
 'Big Data Engineer',
 'Référent en Automatisation de Traitement',
 'IT Data Engineer',
 'Data Engineer',
 'Technicien Data Center',
 'Data Engineer Expérimenté',
 'Consultant BI/BIG DATA',
 'Ingénieur Big Data Confirmé',
 'Consultant BI/BIG DATA',
 'Ingénieur DATA/BI',
 'Data Engineer Transport',
 'Alternance Tracking & Data Analyse',
 'Data Engineer Banque',
 'Consultant ESB DATA',
 'Data Engineer Confirmé',
 'Consultant Stratégie Data',
 "Directeur d'Agence (RGPD)",
 'Data Analyst',
 "Directeur d'Agence (RGPD)",
 'Data Engineer',
 'Data Manager',
 "Directeur d'Agence (RGPD)",
 'Développeur Big Data',
 'Développeur Big Data',
 'Data Engineer Senior',
 'Consultant MOE BI Orienté DATA',
 "Directeur d'Agence (RGPD)",
 'Data Scientist Sécurité/Population',
 'Architect Data Expérimenté',
 'Data Manager',
 'Data Analyst',
 'Développeur Big Data',
 'Développeur Big Data',
 'Data Manager',
 "Directeur d'Agence (RGPD)",
 'Data Manager',
 'Data Scientist',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
 "Directeur d'Agence (RGPD)",
 'Data Scientist',
 'Data Analyst',
 'Développeur Big Data',
 'Développeur Big Data',
 'Développeur Big Data',
    'Développeur Big Data',
 "Directeur d'agence (RGPD)",
 'Développeur Big Data',
 'Développeur Big Data',
 'Data scientist',
 'Consultant Data',
 "Directeur d'agence (RGPD)",
 'Data Engineer',
 "Directeur d'agence (RGPD)",
 'Data manager',
 'Data analyst',
 'Data analyst',
 'Data analyst',
 'Data analyst',
 'Data analyst',
 'Data steward',
 'Technicien Data Center',
 'Technicien Data Center',
 'Ingénieur data scientist',
 'Contrôleur de Gestion Data',
 'Ingénieur Data',
 'Chef de projet Data',
 'Ingénieur Big Data',
 'Data Engineer',
 'Data Manager',
 'Data scientist',
 "Directeur d'agence (RGPD)",
 'Data scientist',
 'Data analyst',
 'Data Manager',
 'Data analyst',
 'Data Manager',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Data Engineer',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Ingénieur Data',
 'Data Engineer',
    'Data manager',
 'Data analyst',
 "Directeur d'agence (RGPD)",
 'Contrôleur de gestion Data',
 'Data protection officer',
 'Consultant SIG/DATA',
 'Responsable de suivi des programmes',
 "Directeur d'agence (RGPD)",
 'Data Analyst',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Chargé de mission Protection des données',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Ingénieur Logiciel Data',
 "Directeur d'agence (RGPD)",
 'Consultant Data Engineer',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Architecte data',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Assistant protection des données',
    "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'DATA ANALYST',
 "Directeur d'agence (RGPD)",
 'DATA Scientist',
 "Directeur d'agence (RGPD)",
 'Data Web Analyst',
 'Contrôleur de gestion Data',
 'Data Engineer',
 'Chef de Projet MOA Data',
 'Contrôleur de Gestion Data Risque',
 'Data Analyst',
 'Manager Data Strategy',
 "Directeur d'agence (RGPD)",
 'Ingénieur Systèmes Big Data',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Data Engineer',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'Juriste / DPO mutualisé',
 "WEB ANALYST",
 'Alternant.e Méthode Maintenance',
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 "Directeur d'agence (RGPD)",
 'DATA ENGINEERING',
 'Data Architect',
 'Data analyste informatique',
    'Chef de projet informatique',
 'Consultant Data',
 'Architecte informatique',
 'Data Analyst',
 'Consultant Big Data',
 'Tech Lead Big Data',
 'Ingénieur Big Data',
 'Data Engineer',
 'Data Engineer',
 'Data Engineer',
 'Consultant Data migration',
 'Data Center Architect',
 'Data Manager',
 'Data Engineer',
 'Expert Data Cloud',
 'Data Engineer',
 'Data Analyst',
 'Chef de projet Data',
 'Data Engineer',
 'Data Centre Architect',
 'Data Engineer',
 'Data Ingénieur',
 'Data Expert',
 'Consultant Data Analyst',
 'Data Centre Architect',
 'Tech Lead Data',
 'Azure Data Engineer',
 'Data Engineer',
 'Data Analyst',
 'Data Architect',
 'Responsable Data',
 'Gestionnaire CRM',
 'Lead Data Engineer',
 'Data Scientist',
 'Junior Data Scientist',
 'Ingénieur Data',
 'Data Scientist',
 'Data Analyst',
 'Consultant Data',
 'Data Engineer',
 'Développeur Big Data',
 'Consultant Data',
 'Data Scientist',
 'PMO Data',
 'Junior Data Scientist',
 'Consultant Data',
 'Data Analyst',
 'Chef de projet informatique',
 'Data Scientist',
 'Développeur Java'
]

In [146]:
len(simplified_job_titles)

1800

In [147]:
# Créer un DataFrame à partir de la liste
df_job2_clean = pd.DataFrame(simplified_job_titles, columns=["job_titles_clean"])

# Sauvegarder le DataFrame au format CSV
df_job2_clean.to_csv("../data/df_job2_clean.csv", index=False)

In [148]:
df_job_clean = pd.read_csv("../data/df_job_clean.csv")
df_job_clean.shape

(500, 1)

In [149]:
df_job2_clean = pd.read_csv("../data/df_job2_clean.csv")
df_job2_clean.shape

(1800, 1)

In [150]:
df_job3_clean = pd.concat([df_job_clean, df_job2_clean], axis=0)
df_job3_clean.shape

(2300, 1)

In [151]:
# Sauvegarder le DataFrame au format CSV
df_job3_clean.to_csv("../data/df_job3_clean.csv", index=False)